<a href="https://colab.research.google.com/github/DanaJian/Machine-learning-technologies/blob/main/Analysis_and_prediction_of_time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [174]:
import pandas as pd
import numpy as np

from datetime import datetime

import plotly.graph_objects as go
import plotly.subplots as sp

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sma

from scipy.stats import boxcox

import warnings
warnings.filterwarnings('ignore')

1. ЗАГРУЗКА ДАННЫХ

In [175]:
data = pd.read_csv('/content/Time_Series.csv', index_col=0)
data

,came
date,
2016-01-10,1345
2016-01-17,2066
2016-01-24,1979
2016-01-31,1909
2016-02-07,1575
...,...
2019-03-03,1483
2019-03-10,1172
2019-03-17,1435


In [176]:
data.dtypes

came    int64
dtype: object

2. ПРОВЕДЕНИЕ ТЕСТА ДИКИ-ФУЛЛЕРА

In [177]:
result = adfuller(data['came'])

if result[1] > 0.05:
    print("p-value:", result[1])
    print("Нулевая гипотеза не отвергается: ряд не стационарен")
else:
    print("p-value:", result[1])
    print("Нулевая гипотеза отвергается: ряд стационарен")
    
print("ADF статистика:", result[0])

if result[0] < result[4]['1%']:
    print("Критическое значение -1%:", result[4]['1%'])
    print("Альтернативная гипотеза: ряд стационарен")
elif result[0] < result[4]['5%']:
    print("Критическое значение -5%:", result[4]['5%'])
    print("Альтернативная гипотеза: ряд стационарен")
else:
    print("Критическое значение -10%:", result[4]['10%'])
    print("Альтернативная гипотеза: ряд не стационарен")


p-value: 0.6320331972965831
Нулевая гипотеза не отвергается: ряд не стационарен
ADF статистика: -1.2937360199060157
Критическое значение -10%: -2.57625492712671
Альтернативная гипотеза: ряд не стационарен


3. РЕАЛИЗАЦИЯ ДЕКОМПОЗИЦИИ ВРЕМЕННОГО РЯДА

In [178]:
period = 12
decomp = sm.tsa.seasonal_decompose(data, period=period)

fig = sp.make_subplots(rows=4, cols=1)
fig.add_trace(go.Scatter(x=data.index, y=data['came'], name='Временной ряд'), row=1, col=1)
fig.update_yaxes(title_text="Временной ряд", row=1, col=1)
fig.add_trace(go.Scatter(x=data.index, y=decomp.trend, name='Тренд'), row=2, col=1)
fig.update_yaxes(title_text="Тренд", row=2, col=1)
fig.add_trace(go.Scatter(x=data.index, y=decomp.seasonal, name='Сезонность'), row=3, col=1)
fig.update_yaxes(title_text="Сезонность", row=3, col=1)
fig.add_trace(go.Scatter(x=data.index, y=decomp.resid, name='Шум'), row=4, col=1)
fig.update_yaxes(title_text="Шум", row=4, col=1)
fig.update_layout(title='Декомпозиция временного ряда', height=1000)
fig.show()

4. ПРЕОБРАЗОВАНИЕ БОКСА-КОКСА

In [179]:
lamda = boxcox(data.came)[1]
print("Оптимальное значение lamda для преобразования: ", lamda)
data['boxcox_came'] = boxcox(data.came, lamda)
fig = sp.make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=data.index, y=data.came), row=1, col=1)
fig.update_yaxes(title_text="Временной ряд", row=1, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data.boxcox_came), row=2, col=1)
fig.update_yaxes(title_text="Преобразованнный ряд", row=2, col=1)
fig.update_layout(title='Преобразование Бокса-Кокса', height=1000)
fig.show()

Оптимальное значение lamda для преобразования:  0.11738125874670109


In [180]:
result = adfuller(data.boxcox_came)

if result[1] > 0.05:
    print("p-value:", result[1])
    print("Нулевая гипотеза не отвергается: ряд не стационарен")
else:
    print("p-value:", result[1])
    print("Нулевая гипотеза отвергается: ряд стационарен")
    
print("ADF статистика:", result[0])

if result[0] < result[4]['1%']:
    print("Критическое значение -1%:", result[4]['1%'])
    print("Альтернативная гипотеза: ряд стационарен")
elif result[0] < result[4]['5%']:
    print("Критическое значение -5%:", result[4]['5%'])
    print("Альтернативная гипотеза: ряд стационарен")
else:
    print("Критическое значение -10%:", result[4]['10%'])
    print("Альтернативная гипотеза: ряд не стационарен")

p-value: 0.5348617481593065
Нулевая гипотеза не отвергается: ряд не стационарен
ADF статистика: -1.497269287644376
Критическое значение -10%: -2.57619681359045
Альтернативная гипотеза: ряд не стационарен


5. ОБУЧЕНИЕ МОДЕЛИ ARIMA И ПРЕДСКАЗАНИЕ НА 12 ОТСЧЕТОВ ВПЕРЕД

In [181]:
model = ARIMA(data.came, order=(p, d, q))
results = model.fit()
forecast = results.forecast(steps=12)

In [182]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data.came, name='Временной ряд'))
fig.add_trace(go.Scatter(x=forecast.index, y=forecast.values, name="Спрогнозированные данные", mode='lines', line=dict(color='red')))
fig.update_layout(title='Прогноз на 12 отсчетов вперед')
fig.show()

6. ВЫЧИСЛЕНИЕ МЕТРИК MAPE, SMAPE И MAE

In [183]:
data = data.drop('boxcox_came', axis=1)
train = data.loc['2016-01-10':'2018-03-04']
test = data.loc['2018-03-11':]

In [184]:
model = ARIMA(train.came, order=(p, d, q))
results = model.fit()
forecast = results.forecast(steps=len(test))

In [186]:
metrics = {
    'mae': np.mean(np.abs(forecast.values - test.came)),
    'mape': np.mean(np.abs((test.came - forecast.values) / test.came)) * 100,
    'smape': np.mean(2 * np.abs(forecast.values - test.came) / (np.abs(test.came) + np.abs(forecast.values))) * 100
}

metrics_df = pd.DataFrame(metrics, index=['Значение'])
metrics_df

,mae,mape,smape
Значение,165.002504,12.215951,11.268763
